In [8]:
#test

import torch
from torch import nn
from utils import *
import torch.nn.functional as F
from math import sqrt
from itertools import product as product
import torchvision 
# torchvision은 computer vision에 쓰이는 유명한 데이터셋 또는 모델구조로 이루어진라이브러리

input_image=torch.randn(1,3,300,300,dtype=torch.float)
print(input_image.shape)
conv1=nn.Conv2d(3,64,3,1)
m_pool1=nn.MaxPool2d(3,2)

featuremap1=conv1(input_image)
print(featuremap1)

featuremap2=m_pool1(featuremap1)
print(featuremap2)

torch.Size([1, 3, 300, 300])
tensor([[[[-0.6617, -0.5950,  1.2352,  ..., -1.0478, -0.7456, -0.6908],
          [-0.5109,  1.0194,  1.2959,  ..., -0.2248, -0.3352,  0.5370],
          [ 0.2582,  0.0830, -0.4413,  ..., -0.1925, -0.5819, -0.0752],
          ...,
          [-0.3018,  0.9111, -0.8639,  ...,  0.1740, -0.7938,  0.9567],
          [-0.0832, -0.7155, -0.1707,  ...,  0.3839, -0.3031,  0.5504],
          [ 0.1742,  0.0588, -0.6667,  ..., -0.4110,  0.6283,  0.9882]],

         [[ 0.2095,  0.4294, -0.5024,  ..., -0.1743, -0.1935,  0.9334],
          [ 0.1596, -0.6172,  0.4122,  ...,  0.0947, -0.6789, -0.2404],
          [-0.2524,  0.4133, -0.5945,  ..., -0.1940, -0.0347, -0.1508],
          ...,
          [-0.8809, -0.0827,  0.5982,  ...,  0.7497, -0.6153, -0.0057],
          [-0.8035,  0.3151,  0.2911,  ...,  0.2128,  0.3538, -0.1814],
          [-0.5578, -0.4123,  0.4541,  ..., -0.4153,  0.5872,  0.9152]],

         [[ 0.4656,  0.2023, -0.9175,  ..., -1.0820,  0.0553,  0.3005],
 

In [46]:
class vgg_base(nn.Module):
    def _init_(self):
        self.conv1_1=nn.Conv2d(3,64,3,padding=1) # stride는 default로 1로 지정되어있음
        self.conv1_2=nn.Conv2d(64,64,3,padding=1)
        self.pool1=nn.MaxPool2d(2,2)
        
        self.conv2_1=nn.Conv2d(64,128,3,padding=1)
        self.conv2_2=nn.Conv2d(128,128,3,padding=1)
        self.pool2=nn.MaxPool2d(2,2)
        
        self.conv3_1=nn.Conv2d(128,256,3,padding=1)
        self.conv3_2=nn.Conv2d(256,256,3,padding=1)
        self.conv3_3=nn.Conv2d(256,256,3,padding=1)
        self.pool3=nn.MaxPool2d(2,2,1)
        
        self.conv4_1=nn.Conv2d(256,512,3,padding=1)
        self.conv4_2=nn.Conv2d(512,512,3,padding=1)
        self.conv4_3=nn.Conv2d(512,512,3,padding=1)
        self.pool4=nn.MaxPool2d(2,2)
        
#         self.conv5_1=nn.Conv2d(512,512,3,padding=1)
#         self.conv5_2=nn.Conv2d(512,512,3,padding=1)
#         self.conv5_3=nn.Conv2d(512,512,3,padding=1)
#         self.pool5=nn.MaxPool2d(2,2,1)
        
        self.conv6=nn.Conv2d(512,1024,3)
        self.conv7=nn.Conv2d(1024,1024,3)
        
    def forward(self,inputdata):
        print("input size",inputdata.shape)
        a=F.relu(self.conv1_1(inputdata))
        a=F.relu(self.conv1_2(a))
        a=self.pool1(a)
        print("1:",a.shape)
        
        a=F.relu(self.conv2_1(a))
        a=F.relu(self.conv2_2(a))
        a=self.pool2(a)
        print("2:",a.shape)
        
        a=F.relu(self.conv3_1(a))
        a=F.relu(self.conv3_2(a))
        a=F.relu(self.conv3_3(a))
        a=self.pool3(a)
        conv3_features=a
        print("3:",a.shape)
        
        a=F.relu(self.conv4_1(a))
        a=F.relu(self.conv4_2(a))
        a=F.relu(self.conv4_3(a))
        print("4_3:",a.shape)
        a=self.pool4(a)
        conv4_features=a
        
        a=F.relu(self.conv6(a))
        a=F.relu(self.conv7(a))
        
        
#         a=F.relu(self.conv5_1(a))
#         a=F.relu(self.conv5_2(a))
#         a=F.relu(self.conv5_3(a))
#         a=self.pool5(a)
#         conv5_features=a
#         print("5:",a.shape)
        return a
    
vgg=vgg_base()
vgg._init_()
print(vgg.forward(input_image).shape)

input size torch.Size([1, 3, 300, 300])
1: torch.Size([1, 64, 150, 150])
2: torch.Size([1, 128, 75, 75])
3: torch.Size([1, 256, 38, 38])
4_3: torch.Size([1, 512, 38, 38])
torch.Size([1, 512, 19, 19])
